In [ ]:
from pathlib import Path

path = Path("./data")


[PosixPath('data/dara_pose_r_cam_u_lincheng'), PosixPath('data/data_pose_r_cam_u_lincheng2'), PosixPath('data/data_pose_r_cam_u_3'), PosixPath('data/data_pose_r_cam_u_1'), PosixPath('data/data_ss_2'), PosixPath('data/data_pose_r_cam_u_lincheng'), PosixPath('data/data_ss_'), PosixPath('data/data_pose_r_cam_u_lin'), PosixPath('data/data_pose_r_cam_u_2'), PosixPath('data/dara_pose_r_cam_u_lin'), PosixPath('data/resources'), PosixPath('data/data_pose_r_cam_u_4'), PosixPath('data/data_pose_r_cam_u_lincheng3')]


In [13]:
import json
from pathlib import Path
from collections import OrderedDict

file_contents = {}
for json_f in path.rglob("sa*.json"):
    json_contents = json.loads(json_f.read_text())
    print(len(json_contents))
    file_contents.update(json_contents)
    # break

# Sort the file_contents by the 'prompt' field
sorted_contents = OrderedDict(
    sorted(file_contents.items(), key=lambda item: int(item[1]['prompt']))
)

output = Path("./data") / "lincheng"
output.mkdir(exist_ok=True)
with open(output / "saved_data.json", "w") as f:
    json.dump(sorted_contents, f, indent=4)


11
12
15
6
1
1
81
10
10
13
2


# yolo finetun using different dataset

In [4]:
from tqdm import tqdm

from ultralytics.utils.checks import check_requirements
from ultralytics.utils.downloads import download
from ultralytics.utils.ops import xyxy2xywhn

import numpy as np
from pathlib import Path

check_requirements(('pycocotools>=2.0',))
from pycocotools.coco import COCO

# Make Directories
dir = Path('/home/capre/disk_4/yutao/ultralytics/datasets/Objects365')  # dataset root dir
for p in 'images', 'labels':
    (dir / p).mkdir(parents=True, exist_ok=True)
    for q in 'train', 'val':
        (dir / p / q).mkdir(parents=True, exist_ok=True)

# Train, Val Splits
for split, patches in [('train', 50 + 1), ('val', 43 + 1)]:
    print(f"Processing {split} in {patches} patches ...")
    images, labels = dir / 'images' / split, dir / 'labels' / split

    # Download
    url = f"https://dorc.ks3-cn-beijing.ksyun.com/data-set/2020Objects365%E6%95%B0%E6%8D%AE%E9%9B%86/{split}/"
    if split == 'train':
        download([f'{url}zhiyuan_objv2_{split}.tar.gz'], dir=dir)  # annotations json
        download([f'{url}patch{i}.tar.gz' for i in range(patches)], dir=images, curl=True, threads=8)
    elif split == 'val':
        download([f'{url}zhiyuan_objv2_{split}.json'], dir=dir)  # annotations json
        download([f'{url}images/v1/patch{i}.tar.gz' for i in range(15 + 1)], dir=images, curl=True, threads=8)
        download([f'{url}images/v2/patch{i}.tar.gz' for i in range(16, patches)], dir=images, curl=True, threads=8)

    # Move
    for f in tqdm(images.rglob('*.jpg'), desc=f'Moving {split} images'):
        f.rename(images / f.name)  # move to /images/{split}

    # Labels
    coco = COCO(dir / f'zhiyuan_objv2_{split}.json')
    names = [x["name"] for x in coco.loadCats(coco.getCatIds())]
    for cid, cat in enumerate(names):
        catIds = coco.getCatIds(catNms=[cat])
        imgIds = coco.getImgIds(catIds=catIds)
        for im in tqdm(coco.loadImgs(imgIds), desc=f'Class {cid + 1}/{len(names)} {cat}'):
            width, height = im["width"], im["height"]
            path = Path(im["file_name"])  # image filename
            try:
                with open(labels / path.with_suffix('.txt').name, 'a') as file:
                    annIds = coco.getAnnIds(imgIds=im["id"], catIds=catIds, iscrowd=None)
                    for a in coco.loadAnns(annIds):
                        x, y, w, h = a['bbox']  # bounding box in xywh (xy top-left corner)
                        xyxy = np.array([x, y, x + w, y + h])[None]  # pixels(1,4)
                        x, y, w, h = xyxy2xywhn(xyxy, w=width, h=height, clip=True)[0]  # normalized and clipped
                        file.write(f"{cid} {x:.5f} {y:.5f} {w:.5f} {h:.5f}\n")
            except Exception as e:
                print(e)


Processing train in 51 patches ...
Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/zhiyuan_objv2_train.tar.gz to /home/capre/disk_4/yutao/ultralytics/datasets/Objects365...
Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train/patch2.tar.gz to /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train...
Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train/patch8.tar.gz to /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train...
Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train/patch14.tar.gz to /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train...
Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train/patch10.tar.gz to /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train...
Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/train/patch4.tar.gz to /home/capre/disk_4/yutao/ultr

Moving train images: 1742289it [03:01, 9609.13it/s] 


loading annotations into memory...
Done (t=43.45s)
creating index...
index created!


Class 365/365 Table Tennis : 100%|██████████| 235/235 [00:00<00:00, 9085.19it/s]

Processing val in 44 patches ...


...

curl: (18) transfer closed with 482859222 bytes remaining to read


Unzipping /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/val/patch29.tar.gz to /home/capre/disk_4/yutao/ultralytics/datasets/Objects365/images/val...


Moving val images: 80000it [00:09, 8631.40it/s] 


loading annotations into memory...
Done (t=2.35s)
creating index...
index created!


Class 365/365 Table Tennis : 100%|██████████| 20/20 [00:00<00:00, 8331.95it/s]


In [ ]:
import os
from ultralytics import YOLO
# os.chdir("/home/capre/disk_4/yutao/ultralytics")
# Load a model
model = YOLO("yolo11x-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="Objects365.yaml", epochs=100, imgsz=640)


# yolo finetune for breast seg

In [5]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11x-seg.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="/home/capre/disk_4/yutao/breast-seg/dataset3/dataset.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.38 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.22 🚀 Python-3.11.10 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4090 D, 24202MiB)
engine/trainer: task=segment, mode=train, model=yolo11x-seg.pt, data=/home/capre/disk_4/yutao/breast-seg/dataset3/dataset.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: proyts2017 (proyts2017-nah). Use `wandb login --relogin` to force relogin


Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


train: Scanning /home/capre/disk_4/yutao/breast-seg/dataset3/labels/train... 3463 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3463/3463 [00:12<00:00, 282.63it/s]

train: WARNING ⚠️ /home/capre/disk_4/yutao/breast-seg/dataset3/images/train/0505.png: 1 duplicate labels removed


...

                   all        866       2840      0.489      0.488      0.467      0.426      0.523      0.412      0.416      0.341



100 epochs completed in 2.250 hours.
Optimizer stripped from runs/segment/train6/weights/last.pt, 124.9MB
Optimizer stripped from runs/segment/train6/weights/best.pt, 124.9MB

Validating runs/segment/train6/weights/best.pt...
Ultralytics 8.3.22 🚀 Python-3.11.10 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 4090 D, 24202MiB)
YOLO11x-seg summary (fused): 491 layers, 62,095,683 parameters, 0 gradients, 319.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 28/28 [00:04<00:00,  5.76it/s]


                   all        866       2840      0.416      0.488      0.477      0.436      0.395      0.462      0.445      0.365
                person        176        286      0.648      0.608      0.616      0.545      0.664      0.612      0.618      0.471
                  bird          1          1          0          0      0.142      0.142          0          0      0.142      0.128
                   dog          1          1          0          0      0.249      0.224          0          0      0.249      0.199
                   cow          2          2       0.37        0.5      0.745      0.723      0.376        0.5       0.52       0.52
              backpack          6          6      0.552       0.42      0.445      0.441      0.541      0.402        0.4      0.356
              umbrella          3          7      0.178      0.286      0.175      0.119      0.189      0.286      0.175      0.135
               handbag          6          9      0.473          1   

lr/pg0,█████▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▂▂▂▂▂▂▂▂▂▁
lr/pg1,▃███▇▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▂▂▂▂▂▂▁▁▁
lr/pg2,▃▆███▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
metrics/mAP50(B),▁▁▃▅▅▆▆▆▇▇█▇█▇▆▆▆▇▇▇▇██▇▇██▇█▇▇▇▇▇▇▇▇▇▇▇
metrics/mAP50(M),▁▂▁▃▄▄▅▅▇▆▆▇▆▆▆▇█▇███▆█▇█████▇▇▇▇██▇▇▆▇▆
metrics/mAP50-95(B),▁▃▃▄▆▅▆▅▆▆▇▆███▇▇▇▇██▇█▇████████▇███████
metrics/mAP50-95(M),▂▁▂▃▃▅▅▅▅▆▇▇▇▆▆▇▆▇▆▇▆▇▇▇▇██▇▇█▇▇▇▇▇▇▇▇▇█
metrics/precision(B),▃▁▃▄▄▂▁▂▂▂▁▅▄▆▁▄▂▅▃▅▆▁▂▃▆▅▄█▃▄▂▂▂▃▂▄▄▅▃▃
metrics/precision(M),█▄▆▅▄▁▅▄▃▄▃▅▅▄▄▄▆▅▃▄▅▅▅▄▄▄▄▅▅▅▃▅▄▅▄▃▅▅▅▃
metrics/recall(B),▁▃▄▄▅▆▆▇▆▆▆▄▆▆▆▅▆█▇▆█▇▆█▇▆▇▆▅▅▆▆▇▇▇▆▆▆▇▆
metrics/recall(M),▃▁▁▃▃▅▅█▆▆▇▆▆▇▇▇▆▆▅▅▇▇▆█▇▅▅▆▆▆▅█▆▇█▆▇▆▇▆


# masks filename align

In [4]:
import os

def add_leading_zeros_to_masks(masks_dir, total_digits=4):
    """
    将 masks_dir 中的文件名加上前导零，使其成为指定长度的数字字符串。

    参数：
    - masks_dir: 掩码文件夹的路径
    - total_digits: 文件名应达到的总位数，默认为4
    """
    for filename in os.listdir(masks_dir):
        # 分离文件名和扩展名
        basename, extension = os.path.splitext(filename)
        try:
            # 将文件名转换为整数，以确保文件名是数字
            number = int(basename)
            # 格式化新的文件名，添加前导零
            new_basename = f"{number:0{total_digits}d}"
            new_filename = new_basename + extension
            # 构建完整的源和目标路径
            src = os.path.join(masks_dir, filename)
            dst = os.path.join(masks_dir, new_filename)
            # 重命名文件
            os.rename(src, dst)
            print(f"重命名：{filename} -> {new_filename}")
        except ValueError:
            print(f"跳过非数字文件名：{filename}")

# 使用示例：
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'  # 替换为您的 masks 文件夹路径
add_leading_zeros_to_masks(masks_dir)


重命名：0.png -> 0000.png
重命名：1.png -> 0001.png
重命名：10.png -> 0010.png
重命名：100.png -> 0100.png
重命名：1000.png -> 1000.png
重命名：1001.png -> 1001.png
重命名：1002.png -> 1002.png
重命名：1003.png -> 1003.png
重命名：1004.png -> 1004.png
重命名：1005.png -> 1005.png
重命名：1006.png -> 1006.png
重命名：1007.png -> 1007.png
重命名：1008.png -> 1008.png
重命名：1009.png -> 1009.png
重命名：101.png -> 0101.png
重命名：1010.png -> 1010.png
重命名：1011.png -> 1011.png
重命名：1012.png -> 1012.png
重命名：1013.png -> 1013.png
重命名：1014.png -> 1014.png
重命名：1077.png -> 1077.png
重命名：1078.png -> 1078.png
重命名：1079.png -> 1079.png
重命名：108.png -> 0108.png
重命名：1080.png -> 1080.png
重命名：1081.png -> 1081.png
重命名：1082.png -> 1082.png
重命名：1083.png -> 1083.png
重命名：1084.png -> 1084.png
重命名：1085.png -> 1085.png
重命名：1086.png -> 1086.png
重命名：1087.png -> 1087.png
重命名：1088.png -> 1088.png
重命名：1089.png -> 1089.png
重命名：109.png -> 0109.png
重命名：1090.png -> 1090.png
重命名：1091.png -> 1091.png
重命名：1092.png -> 1092.png
重命名：1093.png -> 1093.png
重命名：1094.png -> 1094.png
重命名：1095.png

# yolo dataset make

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import shutil

# Define paths
images_dir = '/home/capre/disk_4/yutao/breast-seg/images'  # Replace with your images directory
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'      # Replace with your masks directory
dataset_root = '/home/capre/disk_4/yutao/breast-seg/dataset2'  # Replace with your dataset root directory

# Create necessary directories
os.makedirs(os.path.join(dataset_root, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'val'), exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle and split into train and val
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

def process_dataset(phase, files):
    for image_file in files:
        # Read image and mask
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)  # Assuming mask has the same name
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if mask exists
        if mask is None:
            print(f"Mask not found for image {image_file}")
            continue

        height, width = mask.shape
        # Threshold mask to binary
        _, mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

        # Find contours
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create label file
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(dataset_root, 'labels', phase, label_file)

        with open(label_path, 'w') as f:
            for contour in contours:
                # Simplify contour
                epsilon = 0.001 * cv2.arcLength(contour, True)
                contour = cv2.approxPolyDP(contour, epsilon, True)

                # Flatten contour array
                contour = contour.squeeze()
                if contour.ndim != 2:
                    continue  # Skip if contour is not 2D

                # Normalize coordinates
                normalized_contour = contour.astype(np.float32)
                normalized_contour[:, 0] /= width
                normalized_contour[:, 1] /= height

                # Flatten and convert to list
                contour_list = normalized_contour.flatten().tolist()

                # Write to file (class index is 0)
                line = '0 ' + ' '.join(map(str, contour_list))
                f.write(line + '\n')

        # Copy image to dataset folder
        shutil.copy(image_path, os.path.join(dataset_root, 'images', phase, image_file))

        # Visualization (optional)
        # if random.random() < 0.05:  # Adjust the probability as needed
        #     # Plot image and contours
        #     plt.figure(figsize=(10, 10))
        #     plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        #     for contour in contours:
        #         contour = contour.squeeze()
        #         plt.plot(contour[:, 0], contour[:, 1], linewidth=2)
        #     plt.title(f"{phase} - {image_file}")
        #     plt.axis('off')
        #     plt.show()

# Process train and val datasets
process_dataset('train', train_files)
process_dataset('val', val_files)

# Generate dataset YAML file
dataset_yaml = os.path.join(dataset_root, 'dataset.yaml')
with open(dataset_yaml, 'w') as f:
    f.write(f"path: {dataset_root}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: \n\n")
    f.write("names:\n")
    f.write("  0: object\n")


# add label from predict

In [4]:
import os
import cv2
import numpy as np
import random
import shutil
from ultralytics import YOLO

# Define paths
images_dir = '/home/capre/disk_4/yutao/breast-seg/images'  # Replace with your images directory
masks_dir = '/home/capre/disk_4/yutao/breast-seg/masks'      # Replace with your masks directory
dataset_root = '/home/capre/disk_4/yutao/breast-seg/dataset3'  # Replace with your dataset root directory

# Create necessary directories
os.makedirs(os.path.join(dataset_root, 'images', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'images', 'val'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'train'), exist_ok=True)
os.makedirs(os.path.join(dataset_root, 'labels', 'val'), exist_ok=True)

# Get list of image files
image_files = [f for f in os.listdir(images_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle and split into train and val
random.shuffle(image_files)
split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

# Load the YOLO model
model = YOLO('yolo11x-seg.pt')  # Replace with the correct path to your model

# Get class names and assign a new class ID for your mask label
class_names = model.names  # A dictionary {class_id: class_name}
max_class_id = max(class_names.keys())
mask_class_id = max_class_id + 1
class_names[mask_class_id] = 'breast'  # Replace 'my_mask_class' with your class name

def process_dataset(phase, files):
    for image_file in files:
        # Read image and mask
        image_path = os.path.join(images_dir, image_file)
        mask_path = os.path.join(masks_dir, image_file)  # Assuming mask has the same name
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Check if mask exists
        if mask is None:
            print(f"Mask not found for image {image_file}")
            continue

        original_height, original_width = image.shape[:2]

        # Run the model to get predictions
        results = model.predict(source=image, save=False, verbose=False)
        result = results[0]

        # Create label file
        label_file = os.path.splitext(image_file)[0] + '.txt'
        label_path = os.path.join(dataset_root, 'labels', phase, label_file)

        with open(label_path, 'w') as f:
            # Process predicted masks
            if hasattr(result, 'masks') and result.masks is not None:
                pred_masks = result.masks.data.cpu().numpy()
                pred_classes = result.boxes.cls.cpu().numpy().astype(int)
                for mask_pred, class_id in zip(pred_masks, pred_classes):
                    # Convert mask to binary image
                    mask_pred = (mask_pred > 0.5).astype(np.uint8) * 255

                    # Resize mask back to original image size
                    mask_pred_resized = cv2.resize(mask_pred, (original_width, original_height), interpolation=cv2.INTER_NEAREST)

                    # Find contours
                    contours, _ = cv2.findContours(mask_pred_resized, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                    for contour in contours:
                        # Simplify contour
                        epsilon = 0.001 * cv2.arcLength(contour, True)
                        contour = cv2.approxPolyDP(contour, epsilon, True)

                        # Flatten contour array
                        contour = contour.squeeze()
                        if contour.ndim != 2:
                            continue  # Skip if contour is not 2D

                        # Normalize coordinates
                        normalized_contour = contour.astype(np.float32)
                        normalized_contour[:, 0] /= original_width
                        normalized_contour[:, 1] /= original_height

                        # Flatten and convert to list
                        contour_list = normalized_contour.flatten().tolist()

                        # Write to file
                        line = f"{class_id} " + ' '.join(map(str, contour_list))
                        f.write(line + '\n')
            else:
                print(f"No predicted masks for image {image_file}")

            # Process your own mask
            # Threshold mask to binary
            _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

            # Find contours
            contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            for contour in contours:
                # Simplify contour
                epsilon = 0.001 * cv2.arcLength(contour, True)
                contour = cv2.approxPolyDP(contour, epsilon, True)

                # Flatten contour array
                contour = contour.squeeze()
                if contour.ndim != 2:
                    continue  # Skip if contour is not 2D

                # Normalize coordinates
                normalized_contour = contour.astype(np.float32)
                normalized_contour[:, 0] /= original_width
                normalized_contour[:, 1] /= original_height

                # Flatten and convert to list
                contour_list = normalized_contour.flatten().tolist()

                # Write to file with mask_class_id
                line = f"{mask_class_id} " + ' '.join(map(str, contour_list))
                f.write(line + '\n')

        # Copy image to dataset folder
        shutil.copy(image_path, os.path.join(dataset_root, 'images', phase, image_file))

# Process train and val datasets
process_dataset('train', train_files)
process_dataset('val', val_files)

# Generate dataset YAML file
dataset_yaml = os.path.join(dataset_root, 'dataset.yaml')
with open(dataset_yaml, 'w') as f:
    f.write(f"path: {dataset_root}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write("test: \n\n")
    f.write("names:\n")
    for class_id in sorted(class_names.keys()):
        f.write(f"  {class_id}: {class_names[class_id]}\n")


No predicted masks for image 2417.png
No predicted masks for image 1194.png
No predicted masks for image 1076.png
No predicted masks for image 2074.png
No predicted masks for image 2599.png
No predicted masks for image 0803.png
No predicted masks for image 0003.png
No predicted masks for image 0834.png
No predicted masks for image 3015.png
No predicted masks for image 1363.png
No predicted masks for image 0802.png
No predicted masks for image 4286.png
No predicted masks for image 1867.png
No predicted masks for image 3129.png
No predicted masks for image 0585.png
No predicted masks for image 2842.png
No predicted masks for image 3334.png
No predicted masks for image 3077.png
No predicted masks for image 1082.png
No predicted masks for image 3661.png
No predicted masks for image 2402.png
No predicted masks for image 0391.png
No predicted masks for image 0239.png
No predicted masks for image 0917.png
No predicted masks for image 3145.png
No predicted masks for image 0251.png
No predicted